<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression


In [19]:
df    = pd.read_csv('demanda.csv')

#df.std(numeric_only = True) 
#df.mean(numeric_only = True)
#df = df - df.mean(numeric_only = True)
#df = df / df.std(numeric_only = True)
X   = pd.DataFrame(df[['X0','X1','X2','X3','X4']]).to_numpy()
Y   = pd.DataFrame(df[['Y']]).to_numpy()

In [20]:
print(X)

[[1.0000000e+00 8.4462546e+03 8.5425188e+03 8.6361652e+03 7.6802550e+03]
 [1.0000000e+00 8.4054098e+03 8.5629526e+03 8.6145186e+03 7.6400082e+03]
 [1.0000000e+00 8.3066222e+03 8.6091252e+03 8.6150144e+03 7.6234334e+03]
 ...
 [1.0000000e+00 8.2611958e+03 8.6555832e+03 8.5076230e+03 7.5641578e+03]
 [1.0000000e+00 8.1889102e+03 8.5253702e+03 8.5274594e+03 7.5058454e+03]
 [1.0000000e+00 8.1835672e+03 8.5563534e+03 8.5297884e+03 7.4874990e+03]]


In [21]:
#https://scikit-learn.org/stable/modules/svm.html
#https://scikit-learn.org/stable/auto_examples/svm/plot_svm_regression.html
#https://medium.com/pursuitnotes/support-vector-regression-in-6-steps-with-python-c4569acd062d
from sklearn import svm
model = svm.SVR()
model.fit(X, Y)
model.predict(X)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([8718.76930071, 8718.22310177, 8717.8247226 , 8716.56327533,
       8716.28180043, 8717.74189254, 8718.64617643, 8720.31404185,
       8723.48181413, 8725.25459798, 8727.93472454, 8727.96055912,
       8726.89170426, 8728.57743954, 8729.43836824, 8729.15026381,
       8731.34021447, 8732.24883222, 8732.30728384, 8731.20757056,
       8732.47281204, 8733.51384098, 8733.35801892, 8733.72030975,
       8733.94438312, 8735.50030581, 8735.07430392, 8735.17864264,
       8735.78153031, 8735.87444842, 8737.21249036, 8737.0608361 ,
       8737.24085207, 8738.87449983, 8739.18505548, 8739.84567853,
       8740.65701348, 8741.11553231, 8740.89193715, 8740.19256329,
       8739.31043618, 8737.74981674, 8736.60712885, 8737.55433956,
       8739.49267078, 8739.39479913, 8739.06623326, 8736.47452793,
       8734.5308694 , 8734.25019316, 8733.50361018, 8733.33415291,
       8733.26357301, 8733.62984761, 8732.0540414 , 8732.4754691 ,
       8731.40614492, 8732.40171972, 8730.7342715 , 8728.81739

In [22]:
model = LinearRegression()
model.fit(X, Y)
r_sq = model.score(X, Y)
print('coefficient of determination:', r_sq)
print('intercept:', model.intercept_)
intercept: 5.633333333333329
print('slope:', model.coef_)
y_pred = model.predict(X)
#print('predicted response:', y_pred, sep='\n')

coefficient of determination: 0.8844789855247372
intercept: [534.89141899]
slope: [[0.         0.10455941 0.3784022  0.30081846 0.17503607]]
